In [ ]:
# Databricks notebook source
#%pip install -r ../../data_prep_requirements.txt
#dbutils.library.restartPython()

In [ ]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [ ]:
################################################################################### 
# Data Ingestion Pipeline
#
# This pipeline is designed to process raw documentation data from a specified data source URL. 
# The data is stored in a Unity Catalog within a specified database for later processing.
#
# Parameters:
# * uc_catalog (required)                     - Name of the Unity Catalog containing the input data
# * schema (required)                         - Name of the schema inside the Unity Catalog
# * raw_data_table (required)                 - Name of the raw data table inside the database of the Unity Catalog
# * data_source_url (required)                - URL of the data source. Default is "https://docs.databricks.com/en/doc-sitemap.xml"
# * bundle_root (required)                    - Root of the bundle
#
# Widgets:
# * Unity Catalog: Text widget to input the name of the Unity Catalog
# * Schema: Text widget to input the name of the database inside the Unity Catalog
# * Raw data table: Text widget to input the name of the raw data table inside the database of the Unity Catalog
# * Data Source URL: Text widget to input the URL of the data source
# * Root of bundle: Text widget to input the root of the bundle
#
# Usage:
# 1. Set the appropriate values for the widgets.
# 2. Run the pipeline to collect and store the raw documentation data.
#
##################################################################################

### Widget creation


In [ ]:
# List of input args needed to run this notebook as a job
# Provide them via DB widgets or notebook arguments in your DAB resources

# A Unity Catalog containing the input data
dbutils.widgets.text(
    "uc_catalog",
    "agentops_stacks_dev",
    label="Unity Catalog",
)
# Name of schema
dbutils.widgets.text(
    "schema",
    "agentops",
    label="Schema",
)
# Name of raw data table
dbutils.widgets.text(
    "raw_data_table",
    "raw_documentation",
    label="Raw data table",
)

# Data source url
dbutils.widgets.text(
    "data_source_url",
    "https://docs.databricks.com/en/doc-sitemap.xml",
    label="Data Source URL",
)

# Bundle root
dbutils.widgets.text(
    "bundle_root",
    "/",
    label="Root of bundle",
)

### Define input and output variables


In [ ]:
uc_catalog = dbutils.widgets.get("uc_catalog")
schema = dbutils.widgets.get("schema")
raw_data_table = dbutils.widgets.get("raw_data_table")
data_source_url = dbutils.widgets.get("data_source_url")
bundle_root = dbutils.widgets.get("bundle_root")

assert uc_catalog != "", "uc_catalog notebook parameter must be specified"
assert schema != "", "schema notebook parameter must be specified"
assert raw_data_table != "", "raw_data_table notebook parameter must be specified"
assert data_source_url != "", "data_source_url notebook parameter must be specified"
assert bundle_root != "", "bundle_root notebook parameter must be specified"

# Updating to bundle root
import sys 

root = dbutils.widgets.get("bundle_root")
sys.path.append(root)

### Use the catalog and database specified in the notebook parameters


In [ ]:
spark.sql(f"""CREATE SCHEMA IF NOT EXISTS `{uc_catalog}`.`{schema}`""")

spark.sql(f"""USE `{uc_catalog}`.`{schema}`""")

### Download and store data to UC


In [ ]:
from data_preparation.data_ingestion.ingestion.fetch_data import fetch_data_from_url

if not spark.catalog.tableExists(f"{raw_data_table}") or spark.table(f"{raw_data_table}").isEmpty():
    # Download the data to a DataFrame 
    doc_articles = fetch_data_from_url(spark, data_source_url)

    #Save them as to unity catalog
    doc_articles.write.mode('overwrite').saveAsTable(f"{raw_data_table}")

    doc_articles.display()

In [ ]:
dbutils.notebook.exit(0)